In [235]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
from tables import JPEGProbabilityTable, IllustrativeProbabilityTable

In [237]:
ptable = JPEGProbabilityTable()

In [238]:
ptable.is_exchange_needed, hex(ptable.qe)

(True, '0x5a1d')

In [239]:
ptable = IllustrativeProbabilityTable()
ptable.is_exchange_needed, hex(ptable.qe)

(True, '0xac1')

In [240]:
class Encoder(object):
    
    def __init__(self):
        self.ptable = IllustrativeProbabilityTable()
        self.lps = 1
        self.C = 0
        self.A = self.ptable.one
        self.steps = 0

    @property
    def Qe(self):
        return self.ptable.qe
    
    def renormalize(self):
        while self.A < self.ptable.threequarter:
            self.A <<= 1
            self.C <<= 1
    
    def encode_mps(self):
        self.steps += 1
        self.A -= self.Qe
        if self.A < self.ptable.threequarter:
#             if self.ptable.is_exchange_needed:
#                 self.C += self.A
#                 self.A = self.Qe
            self.renormalize()
        self.ptable.update_mps()
    
    def encode_lps(self):
        self.steps += 1
        self.C += (self.A - self.Qe)
        self.A = self.Qe
#         if self.A >= self.Qe:
#             self.C += self.A
#             self.A = self.Qe
        self.renormalize()
        self.ptable.update_lps()

In [241]:
enc = Encoder()
enc.encode_lps()
enc.encode_mps()
enc.A, enc.C, enc.Qe, enc.ptable.one, enc.steps

(41295, 654992, 2689, 43690, 2)

In [242]:
class Decoder(object):
    
    def __init__(self, C, steps):
        self.ptable = IllustrativeProbabilityTable()
        self.lps = 1
        self.C = C
        self.A = self.ptable.one
        self.steps = steps

    @property
    def Qe(self):
        return self.ptable.qe
    
    def decode(self):
        while self.steps > 0:
            self.steps -= 1
            self.A = self.A - self.Qe
            if (enc.C & (((1 << 7) - 1) << 8)) < self.A:
                # Cx 
                pass

In [243]:
enc.A

41295

In [244]:
enc.C &

SyntaxError: invalid syntax (<ipython-input-244-dee4a9d92699>, line 1)

In [245]:
enc.C & (((1 << 7) - 1) << 8)

32256

In [246]:
enc.Qe

2689